In [39]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv("../data/iris.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
Id               150 non-null int64
SepalLengthCm    150 non-null float64
SepalWidthCm     150 non-null float64
PetalLengthCm    150 non-null float64
PetalWidthCm     150 non-null float64
Species          150 non-null object
dtypes: float64(4), int64(1), object(1)
memory usage: 7.1+ KB


In [4]:
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [159]:
X_data = StandardScaler().fit_transform(df.iloc[:, [1, 2, 3, 4]].values)
X_data.shape, X_data.dtype

((150, 4), dtype('float64'))

In [160]:
y_data = LabelEncoder().fit_transform(df["Species"])
y_data_ohe = OneHotEncoder(sparse = False).fit_transform(y_data.reshape(-1, 1))
print("dtype of y_data: ", y_data_ohe.dtype)
y_data_ohe[np.random.choice(y_data_ohe.shape[0], 10, replace=False), ]

dtype of y_data:  float64


array([[ 0.,  1.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.]])

In [161]:
lr = LogisticRegression()
lr.fit(X_data, y_data)
accuracy_score(y_data, lr.predict(X_data))

0.92666666666666664

In [169]:
steps = 1000
batch_size = 1
num_classes = 3
learning_rate = 0.1
n, d = X_data.shape

X = tf.placeholder(tf.float32, [None, d])
y_true = tf.placeholder(tf.float32, [None, num_classes])
W = tf.Variable(tf.random_normal([d, num_classes], seed = 100))
b = tf.Variable(tf.random_normal([num_classes], seed = 100))
y_pred = tf.nn.softmax(tf.matmul(X, W) + b)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_true * tf.log(y_pred), axis = 1))

solver = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(cross_entropy)

correct_mask = tf.equal(tf.argmax(y_pred, axis = 1), tf.argmax(y_true, axis = 1))
accuracy = tf.reduce_mean(tf.cast(correct_mask, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(steps):
        #batch_xs, batch_ys = data.train.next_batch(batch_size)
        sess.run(solver, feed_dict={X: X_data, y_true: y_data_ohe})

    # Test
    ans = sess.run(accuracy, feed_dict={X: X_data, y_true: y_data_ohe})

print("Accuracy: {:.4}%".format(ans*100))

Accuracy: 97.33%
